In [1]:
import os
from selenium import webdriver
import pandas as pd
import datetime, time, csv
# import gspread
from bs4 import BeautifulSoup

In [2]:
def get_header_columns(header):
    
    '''
    PortfolioVisualizer Import Benchmark. 
    To generate a list of columns from the webpage HTML file from 
    a table, in order to create a Pandas DataFrame.

    '''
    
    soup = BeautifulSoup(header, 'html.parser')

    columns = []
    initials = []
    index = 0
    for row in soup.findAll('th'):
    #     print(row)
    #     print(row.attrs)
        if 'scope' in row.attrs:
            if 'colspan' in row.attrs:
    #             print('colspan', row['class'], get_initials(row.text.split(' ')))
                initials.append(get_initials(row.text.split(' ')))
            else:
                columns.append(row.text)
        if 'class' in row.attrs and 'group-start' in row['class']:
    #         print (initials[index], row.text)
            columns.append(initials[index]+'_'+row.text)

        if 'class' in row.attrs and 'group-end' in row['class']:
    #         print (initials[index], row.text)
            columns.append(initials[index]+'_'+row.text)
            index += 1
        
    return columns

In [3]:
def get_initials(string_list=[]):
    return "".join(item[0].upper() for item in string_list)

In [4]:
##### get PV name and password

import getpass

PVNAME = getpass.getpass("Username:")
print (PVNAME)
PVPASSWD = getpass.getpass("Password for " + PVNAME + ":")

Username:········

Password for :········


# Create Benchmarks

In [4]:
os.environ.update({'PVNAME':'scubamut@gmail.com'})

In [5]:
os.environ.get('PVNAME')

'scubamut@gmail.com'

In [6]:
os.environ.update({'PVPASSWD':'9D@3G!#qH5ZK*k^X*L%x'})

In [359]:
# login to PV

# browser = webdriver.Chrome(executable_path="/usr/lib/chromium-browser/chromedriver",
#                            service_args=["--verbose", "--log-path=~\Downloads\qc1.log"])
browser = webdriver.Chrome(executable_path="/usr/local/share/chromedriver",
                           service_args=["--verbose", "--log-path=~\Downloads\qc1.log"])
# browser.set_window_position(-1138,1)
# browser = webdriver.Chrome(executable_path="D:\\chromedriver.exe", service_args=["--verbose", "--log-path=~\Downloads\qc1.log"])
browser.set_window_position(1,1)
# browser.set_window_size(1038, 875)
browser.fullscreen_window()
browser.get('https://www.portfoliovisualizer.com/login')
time.sleep(2)
user = browser.find_element_by_id('username')
user.send_keys(os.environ.get('PVNAME'))
password = browser.find_element_by_id('password')
password.send_keys(os.environ.get('PVPASSWD'))
time.sleep(2)
login = browser.find_element_by_id('submitButton')
login.click()

In [412]:
# add PV assets and year-end, as necessary

assets = {'OOM2000': 'http://bit.ly/2AMTnLO',
          'OOM1987' : 'http://bit.ly/2DmKEgW',
          'ETFMDBS0': 'http://bit.ly/2iN43TU',
          'MDBS00' : 'https://goo.gl/Zzwnhk'} 

# remember that endYear is 2018 so change if necessary

endYear = '2018'

In [462]:
for no,asset in enumerate(assets.keys()):
    print(no,asset,assets[asset])

0 AAA000  http://bit.ly/2Cprx74


In [386]:
for no,asset in enumerate(assets.keys()):

    print(asset, assets[asset])   
    browser.get(assets[asset])

    browser.find_element_by_id('endYear').clear()
    browser.find_element_by_id('endYear').send_keys(endYear)

    browser.find_element_by_id('submitButton').click()
    browser.find_element_by_xpath('//*[@id="tabs"]/li[4]/a').click() 

    monthly_returns = browser.find_element_by_css_selector('#monthlyReturns').text
    results = monthly_returns.splitlines()

    data = [results[2:][i].split(sep=' ') for i in range(len(results[3:]))]

    # find table header
    elem = browser.find_element_by_xpath('//*[@id="monthlyReturns"]/div/table/thead/tr')
    columns = [t.text for t in elem.find_elements_by_tag_name('th')]

    returns = pd.DataFrame(data, columns=columns, index=None)
    returns.Month = [returns.Month.astype(str)[i].zfill(2) for i in range(len(returns))]   
    returns.to_csv('~/Downloads/' + asset + '.csv', index=None)

    returns.loc[:,'Period'] = returns.Year.astype(str)+"/"+returns.Month

    df = pd.DataFrame( {'Period': returns['Period'], 'Return': returns['Timing Portfolio Return']}, index=None)
    df.to_csv('~/Downloads/' + asset + '.csv', index=None)

    data_path = '/home/scubamut/Downloads/'
    symbol = asset

    browser.get('https://www.portfoliovisualizer.com/manage-benchmarks#import')

    # Upload Data File
    browser.find_element_by_id("upload").clear()
    browser.find_element_by_id("upload").send_keys(data_path + symbol +'.csv')

    # Series (Type)
    browser.find_element_by_id('seriesType_chosen').click()
    # choose SeriesType : li[1|2|3|4] eg li[1] gives Monthly Returns
    browser.find_element_by_xpath('//*[@id="seriesType_chosen"]/div/ul/li[1]').click()

    # Series Name
    browser.find_element_by_id("benchmarkName").clear()
    browser.find_element_by_id("benchmarkName").send_keys(asset+'.csv')

    # Assigned Ticker
    browser.find_element_by_id("benchmarkSymbol").clear()
    browser.find_element_by_id("benchmarkSymbol").send_keys(symbol)

    data_path = '/home/scubamut/Downloads/'
    symbol = asset

    browser.get('https://www.portfoliovisualizer.com/manage-benchmarks#import')

    # Upload Data File
    browser.find_element_by_id("upload").clear()
    browser.find_element_by_id("upload").send_keys(data_path + symbol +'.csv')

    # Series (Type)
    browser.find_element_by_id('seriesType_chosen').click()
    # choose SeriesType : li[1|2|3|4] eg li[1] gives Monthly Returns
    browser.find_element_by_xpath('//*[@id="seriesType_chosen"]/div/ul/li[1]').click()

    # Series Name
    browser.find_element_by_id("benchmarkName").clear()
    browser.find_element_by_id("benchmarkName").send_keys(asset+'.csv')

    # Assigned Ticker
    browser.find_element_by_id("benchmarkSymbol").clear()
    browser.find_element_by_id("benchmarkSymbol").send_keys(symbol)

    time.sleep(2)

    # Percentage Values
    browser.find_element_by_id('percentageValues_chosen').click()
    # choose Yes
    browser.find_element_by_xpath('//*[@id="percentageValues_chosen"]/div/ul/li[2]').click()

    browser.find_element_by_id('benchmarkAssetClass_chosen').click()
    browser.find_element_by_xpath('//*[@id="benchmarkAssetClass_chosen"]/div/ul/li[2]').click()

    # Import Data Series
    browser.implicitly_wait(90)
    browser.find_element_by_id("importBenchmarkButton").click()

    print(browser.find_element_by_class_name("alert").text)

    browser.execute_script("window.history.go(-1)")
    time.sleep(2)
           
browser.close()

MDBS00 https://goo.gl/Zzwnhk
Benchmark series 'MDBS00.csv' imported.
OOM1987 http://bit.ly/2DmKEgW
Benchmark series 'OOM1987.csv' imported.
ETFMDBS0 http://bit.ly/2iN43TU
Benchmark series 'ETFMDBS0.csv' imported.
OOM2000 http://bit.ly/2AMTnLO
Benchmark series 'OOM2000.csv' imported.


# HOW TO GET AS STRATEGY RESULTS

In [539]:
# login to PV

# browser = webdriver.Chrome(executable_path="/usr/lib/chromium-browser/chromedriver",
#                            service_args=["--verbose", "--log-path=~\Downloads\qc1.log"])
browser = webdriver.Chrome(executable_path="/usr/local/share/chromedriver",
                           service_args=["--verbose", "--log-path=~\Downloads\qc1.log"])
# browser.set_window_position(-1138,1)
# browser = webdriver.Chrome(executable_path="D:\\chromedriver.exe", service_args=["--verbose", "--log-path=~\Downloads\qc1.log"])
browser.set_window_position(1,1)
# browser.set_window_size(1038, 875)
browser.fullscreen_window()
browser.get('https://www.portfoliovisualizer.com/login')
time.sleep(2)
user = browser.find_element_by_id('username')
user.send_keys(os.environ.get('PVNAME'))
password = browser.find_element_by_id('password')
password.send_keys(os.environ.get('PVPASSWD'))
time.sleep(2)
login = browser.find_element_by_id('submitButton')
login.click()

In [559]:
assets = [["Adaptive Asset Allocation","AAA000","http://bit.ly/2Cprx74"],
["Allocate Smartly's Meta Strategy","AMS000","http://bit.ly/2Rr648c"],
["Accelerating Dual Momentum","ADM000","http://bit.ly/2Ryk5RT"],
["Vigilant Asset Allocation - Aggressive","VAAA00","http://bit.ly/2ClIFdZ"],
["Varadi's Percentile Channels","PC0000","http://bit.ly/2ClDtqL"],
["Protective Asset Allocation","PAA000","http://bit.ly/2RwZaP1"],
["US Risk Parity Trend Following","USRPTF","http://bit.ly/2RwqNrq"],
["Keuning's Generalized Protective Momentum","GPM000","http://bit.ly/2FlCOIb"],
["Protective Asset Allocation - CPR","PAACPR","http://bit.ly/2MWAjlJ"],
["Efficiente Index","EI0000","http://bit.ly/2MYCgOI"],
["Flexible Asset Allocation","FAA000","http://bit.ly/2MYT08c"],
["Classical Asset Allocation - Defensive (left out of strategies)","CAAD00","http://bit.ly/2N209oq"],
["Stoken's Active Combined Asset (ACA) - Monthly","ACAM00","http://bit.ly/2Ne8uWn"],
["Stoken's Active Combined Asset (ACA)","ACA000","http://bit.ly/2WUNJDn"],
["Elastic Asset Allocation - Defensive","EAAD00","http://bit.ly/2MZwegN"],
["Faber's Trinity Portfolio Lite","TPL000","http://bit.ly/2FogXQ6"],
["US Max Diversification","USMD00","http://bit.ly/2FmnL0y"],
["Antonacci's Composite Dual Momentum","CDM000","http://bit.ly/2FiBghN"],
["Varadi's Minimum Correlation Portfolio","MCP000","http://bit.ly/2Fmo6jQ"],
["Faber's Global Tactical Asset Alloc. 13 (GTAA 13)","GTAA13","http://bit.ly/2MZ0yYA"],
["Classical Asset Allocation - Offensive","CAAO00","http://bit.ly/2FmEO2L"],
["Faber's Global Tactical Asset Alloc. - Agg. 3","GTAAA3","http://bit.ly/2FkCVUa"],
["Vigilant Asset Allocation - Balanced","VAAB00","http://bit.ly/2Fyaanl"],
["Antonacci's Global Equities Momentum (GEM)","GEM000","http://bit.ly/2Fm7dpI"],
["PortfolioCharts' Golden Butterfly","GB0000","http://bit.ly/2FkycSk"],
["Robust Asset Allocation - Aggressive","RAAA00","http://bit.ly/2Fm4xs9"],
["US Equal Risk Contribution","USERC0","http://bit.ly/2FmMSQX"],
["US Min Correlation","USMC00","http://bit.ly/2FghQKH"],
["Tactical Permanent Portfolio","TPP000","http://bit.ly/2Fkzblu"],
["Dalio's All-Weather Portfolio","AWP000","http://bit.ly/2FmOM4l"],
["US Max Sharpe","USMS00","http://bit.ly/2FkEnG6"],
["Philosophical Economics' Growth-Trend Timing","PGTT00","http://bit.ly/2FiPxLy"],
["Browne's Permanent Portfolio","PP0000","http://bit.ly/2FlMiDh"],
["Davis' Three Way Model","TWM000","http://bit.ly/2FmGfhF"],
["Glenn's Paired Switching Strategy","PSS000","http://bit.ly/2Fm5cK9"],
["Novell's Tactical Bond Strategy","TBS000","http://bit.ly/2FkFxkW"],
["60/40 Benchmark","BB6000","http://bit.ly/2N0dNsj"],
["Faber's Sector Relative Strength (Sector RS)","SRS000","http://bit.ly/2FiTdgw"],
["Faber's Ivy Portfolio","IP0000","http://bit.ly/2FiHbDx"],
["Kipnis' Defensive Adaptive Asset Allocation","DAAA00","http://bit.ly/2Fmksqd"]]

In [513]:
assets[0][0],assets[0][1],assets[0][2]


('Adaptive Asset Allocation', 'AAA000', 'http://bit.ly/2Cprx74')

In [560]:
header = ['Name', 'Initial Balance','Final Balance','CAGR','Stdev','Best Year','Worst Year',
          'Max. Drawdown','Sharpe Ratio','Sortino Ratio','US Mkt Correlation']
df = pd.DataFrame(index=[a[1] for a in assets],columns=header)

In [561]:
# for asset in assets[:2]:
for asset in assets:
    name, symbol, link = asset[0],asset[1],asset[2],
    print(name, symbol, link)

    browser.get(link)
    metrics = browser.find_element_by_xpath('//*[@id="growthChart"]/table/tbody/tr[1]').text.split(' ')
    metrics[0] = name
    del(metrics[1])
    df.loc[symbol] = metrics
    
browser.close()

Adaptive Asset Allocation AAA000 http://bit.ly/2Cprx74
Allocate Smartly's Meta Strategy AMS000 http://bit.ly/2Rr648c
Accelerating Dual Momentum ADM000 http://bit.ly/2Ryk5RT
Vigilant Asset Allocation - Aggressive VAAA00 http://bit.ly/2ClIFdZ
Varadi's Percentile Channels PC0000 http://bit.ly/2ClDtqL
Protective Asset Allocation PAA000 http://bit.ly/2RwZaP1
US Risk Parity Trend Following USRPTF http://bit.ly/2RwqNrq
Keuning's Generalized Protective Momentum GPM000 http://bit.ly/2FlCOIb
Protective Asset Allocation - CPR PAACPR http://bit.ly/2MWAjlJ
Efficiente Index EI0000 http://bit.ly/2MYCgOI
Flexible Asset Allocation FAA000 http://bit.ly/2MYT08c
Classical Asset Allocation - Defensive (left out of strategies) CAAD00 http://bit.ly/2N209oq
Stoken's Active Combined Asset (ACA) - Monthly ACAM00 http://bit.ly/2Ne8uWn
Stoken's Active Combined Asset (ACA) ACA000 http://bit.ly/2WUNJDn
Elastic Asset Allocation - Defensive EAAD00 http://bit.ly/2MZwegN
Faber's Trinity Portfolio Lite TPL000 http://bit

In [562]:
df

,Name,Initial Balance,Final Balance,CAGR,Stdev,Best Year,Worst Year,Max. Drawdown,Sharpe Ratio,Sortino Ratio,US Mkt Correlation
AAA000,Adaptive Asset Allocation,"$10,000","$253,867",11.38%,9.75%,33.77%,-5.62%,-11.35%,0.86,1.46,0.40
AMS000,Allocate Smartly's Meta Strategy,"$10,000","$2,394,462",12.65%,7.55%,41.77%,-4.50%,-8.99%,1.02,1.84,0.53
ADM000,Accelerating Dual Momentum,"$10,000","$636,455",15.40%,13.77%,65.57%,-5.39%,-20.09%,0.92,1.55,0.44
VAAA00,Vigilant Asset Allocation - Aggressive,"$10,000","$23,987,334",18.01%,12.56%,60.51%,-5.50%,-16.15%,1.03,2.25,0.23
PC0000,Varadi's Percentile Channels,"$10,000","$116,071",8.23%,5.36%,23.55%,-3.51%,-6.57%,0.95,1.60,0.50
PAA000,Protective Asset Allocation,"$10,000","$2,572,748",12.82%,8.32%,37.38%,-5.19%,-10.32%,0.95,1.69,0.41
USRPTF,US Risk Parity Trend Following,"$10,000","$483,677",8.80%,5.73%,26.27%,-2.53%,-8.60%,0.70,1.06,0.55
GPM000,Keuning's Generalized Protective Momentum,"$10,000","$2,429,810",12.68%,8.23%,41.88%,-3.20%,-9.86%,0.95,1.86,0.17
PAACPR,Protective Asset Allocation - CPR,"$10,000","$1,843,190",12.01%,7.50%,31.57%,-2.83%,-13.14%,0.95,1.70,0.41
EI0000,Efficiente Index,"$10,000","$138,859",9.17%,8.07%,31.54%,-9.49%,-13.28%,0.77,1.20,0.55


# SCRAP

In [563]:
browser.close()

In [490]:
metrics[0] = name
metrics

['Adaptive Asset Allocation',
 '1',
 '$10,000',
 '$253,867',
 '11.38%',
 '9.75%',
 '33.77%',
 '-5.62%',
 '-11.35%',
 '0.86',
 '1.46',
 '0.40']

In [491]:
del(metrics[1])

In [492]:
metrics

['Adaptive Asset Allocation',
 '$10,000',
 '$253,867',
 '11.38%',
 '9.75%',
 '33.77%',
 '-5.62%',
 '-11.35%',
 '0.86',
 '1.46',
 '0.40']

In [495]:
pd.Series(metrics,index=header)

Name                  Adaptive Asset Allocation
Initial Balance                         $10,000
Final Balance                          $253,867
CAGR                                     11.38%
Stdev                                     9.75%
Best Year                                33.77%
Worst Year                               -5.62%
Max. Drawdown                           -11.35%
Sharpe Ratio                               0.86
Sortino Ratio                              1.46
US Mkt Correlation                         0.40
dtype: object

In [535]:

df.loc['AAA000']

Name                  NaN
Initial Balance       NaN
Final Balance         NaN
CAGR                  NaN
Stdev                 NaN
Best Year             NaN
Worst Year            NaN
Max. Drawdown         NaN
Sharpe Ratio          NaN
Sortino Ratio         NaN
US Mkt Correlation    NaN
Name: AAA000, dtype: object

In [530]:
[a[1] for a in assets]

['AAA000',
 'AMS000',
 'ADM000',
 'VAAA00',
 'DAA000',
 'PC0000',
 'PAA000',
 'USRPTF',
 'GPM000',
 'PAACPR',
 'EI0000',
 'FAA000',
 'CAAD00',
 'ACAM00',
 'ACA000',
 'EAAD00',
 'TPL000',
 'USMD00',
 'CDM000',
 'MCP000',
 'GTAA13',
 'CAAO00',
 'GTAAA3',
 'VAAB00',
 'GEM000',
 'GB0000',
 'RAAA00',
 'USERC0',
 'USMC00',
 'TPP000',
 'AWP000',
 'USMS00',
 'PGTT00',
 'PP0000',
 'TWM000',
 'PSS000',
 'TBS000',
 'BB6000',
 'SRS000',
 'IP0000',
 'DAAA00']